In [1]:
#

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("../input/feedback-prize-effectiveness/train.csv")

In [26]:
test = pd.read_csv("../input/feedback-prize-effectiveness/test.csv")

In [3]:
pd.set_option('displitay.max_colwidth', 2550)
train.head(5)

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about how this face on Mars is a natural landform or if there is life on Mars that made it. The story is about how NASA took a picture of Mars and a face was seen on the planet. NASA doesn't know if the landform was created by life on Mars, or if it is just a natural landform.",Lead,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a natural landform because I dont think that there is any life on Mars. In these next few paragraphs, I'll be talking about how I think that is is a natural landform",Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform because there is no life on Mars that we have descovered yet,Claim,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The reason why I think it is a natural landform because, nobody live on Mars in order to create the figure. It says in paragraph 9, ""It's not easy to target Cydonia,"" in which he is saying that its not easy to know if it is a natural landform at this point. In all that they're saying, its probably a natural landform.",Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by alieans because they thought that there was life on Mars.,Counterclaim,Adequate


In [27]:
test.head()

,discourse_id,essay_id,discourse_text,discourse_type
0,a261b6e14276,D72CB1C11673,Making choices in life can be very difficult. People often ask for advice when they can not decide on one thing. It's always good to ask others for their advice when making a choice. When you have multiple opinions you have the ability to make the best choice for yourself.,Lead
1,5a88900e7dc1,D72CB1C11673,Seeking multiple opinions can help a person make a better choice,Position
2,9790d835736b,D72CB1C11673,it can decrease stress levels,Claim
3,75ce6d68b67b,D72CB1C11673,a great chance to learn something new,Claim
4,93578d946723,D72CB1C11673,can be very helpful and beneficial.,Claim


In [39]:
!mkdir models_debug
!mkdir models_folded
def kfold_model_inproc(model_type, X, y, Xtest):
    import os
    print("kfold", str(os.getpid()))
    import tensorflow as tf
    from keras import backend as K
    import gc
    from tqdm import tqdm
    import pickle

    import numpy as np
    split = 5
    oof_pred = np.zeros((X.shape[0],))
    test_pred = np.zeros((X.shape[0],split))
    from sklearn.model_selection import StratifiedKFold
    kf = StratifiedKFold(n_splits=split, shuffle=True)
    i_model=0
    all_train_idx = []
    all_valid_idx = []
    all_model_files = []
    all_metrics = {}
    for train_idx, valid_idx in tqdm(kf.split(X, y)):
        Xt = X.iloc[train_idx]
        yt = y.iloc[train_idx]
        Xv = X.iloc[valid_idx]
        yv = y.iloc[valid_idx]
        all_train_idx.append(train_idx)
        all_valid_idx.append(valid_idx)
        model = model_type.create()
        model_type.fit(model, Xt, yt)
        oof_df_fold = model_type.predict(model, Xv)
        metrics = model_type.evaluate(model, Xv, yv)
        for key in metrics:
            if key in all_metrics:
                all_metrics[key]=np.concat((all_metrics[key], metrics[key]), axis = 1)
            else: 
                all_metrics[key]=metrics[key]
        test_df_fold = model_type.predict(model, Xtest)
        foldData = {}
        foldData['X'], foldData['y'],foldData['valid_idx'],foldData['train_idx']=X,y,valid_idx,train_idx
        foldData['oof_df_fold'],foldData['i_model'], foldData['metrics'], foldDat['test_df_fold']=oof_df_fold,i_model, metrics, test_df_fold
        with open(r"models_debug/"+model_type.name()+"_"+str(i_model)+".bin", "wb") as output_file:
            pickle.dump(foldData, output_file)
        oof_pred[valid_idx] = np.reshape(oof_df_fold, (oof_df_fold.shape[0],))
        test_pred[:, i_model]=test_df_fold
        model_file = model_type.save(i_model, model)
        all_model_files.append(model_file)
        i_model+=1
        K.clear_session()
        del model
        gc.collect()
        K.clear_session()
        gc.collect()
        K.clear_session()
    modelTypeData = {}
    modelTypeData['X'], modelTypeData['y'],modelTypeData['all_valid_idx'],modelTypeData['all_train_idx'], modelTypeData['all_model_files']=X,y,all_valid_idx,all_train_idx, all_model_files
    modelTypeData['model_type'], modelTypeData['oof_pred'],modelTypeData['all_metrics'], modelTypeData['test_pred']=model_type, oof_pred, all_metrics, test_pred
    with open(r"models_folded/"+model_type.name()+".bin", "wb") as output_file:
        pickle.dump(modelTypeData, output_file)
    return oof_pred, test_pred

A subdirectory or file models_debug already exists.
A subdirectory or file models_folded already exists.


In [35]:
class MyModel:
    def name(self):
        pass
    def create(self):
        pass
    def fit(self, model, Xt, yt):
        pass
    def predict(self, model, Xt):
        pass
    def evaluate(self, model, x, y):
        pass
    def save(self, i, model):
        pass
    def load(self, i):
        pass

In [41]:
class MyBertModel(MyModel):
    def create(self):
        import tensorflow as tf
        import tensorflow_hub as hub
        import tensorflow_text as text
        bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
        bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
        text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
        preprocessed_text = bert_preprocess(text_input)
        outputs = bert_encoder(preprocessed_text)
        l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
        l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
        model = tf.keras.Model(inputs=[text_input], outputs = [l])
        print(model.summary())
        METRICS = [
          tf.keras.metrics.BinaryAccuracy(name='accuracy'),
          tf.keras.metrics.Precision(name='precision'),
          tf.keras.metrics.Recall(name='recall')
        ]
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)
        return (model, self)
    def name(self):
        return "bert"
    def fit(self, model, Xt, yt):
        model[1].history = model[0].fit(Xt, yt)
    def predict(self, model, Xt):
        model[0].predict(Xt)
    def evaluate(self, model, x, y):
        return model[0].evaluate(x,y)
        